In [1]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
df = pd.read_csv("noAudible/Features_25hz_no_audible.csv")

df = df.drop ('channel_1_medf', axis=1)
df = df.drop ('channel_2_medf', axis=1)
df = df.drop ('channel_3_medf', axis=1)
df = df.drop ('channel_4_medf', axis=1)
df = df.drop ('channel_5_medf', axis=1)
df = df.drop ('channel_6_medf', axis=1)

#df = pd.read_csv("Features_all_no_audible.csv")
#df = df[(df.modeID != 'audible')]
df

C:\Users\Magomed\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


,channel_1_mav,channel_1_mav2,channel_1_rms,channel_1_iav,channel_1_ssi,channel_1_var,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_iemg,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_meanf,userID,sessionID,uttID,modeID
0,0.000254,0.000217,0.000379,0.990989,0.000563,1.439985e-07,0.258692,0.990989,0.000066,326,...,1186.768066,0.444535,1278,2241,1068,66.053648,1,1001,10010001,audible
1,0.000098,0.000072,0.000154,0.256454,0.000062,2.366934e-08,0.067076,0.256454,0.000026,219,...,796.572021,0.443308,794,1497,728,57.514921,1,1001,10010002,audible
2,0.000282,0.000255,0.000402,1.333337,0.000764,1.614573e-07,0.347359,1.333337,0.000073,394,...,1436.483765,0.445901,1534,2726,1316,73.430718,1,1001,10010003,audible
3,0.000908,0.000808,0.001309,4.413536,0.008328,1.713939e-06,1.152483,4.413536,0.000237,405,...,1481.948608,0.445519,1540,2792,1345,77.039314,1,1001,10010004,audible
4,0.000325,0.000312,0.000442,1.779699,0.001071,1.953260e-07,0.464230,1.779699,0.000085,457,...,1674.552032,0.446329,1716,3143,1523,77.906715,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.000158,0.000123,0.000245,0.429725,0.000163,5.991564e-08,0.112283,0.429725,0.000041,227,...,563.251404,0.375292,1702,2214,389,59.203804,8,8019,80190246,whisper
4796,0.000046,0.000031,0.000074,0.102209,0.000012,5.519670e-09,0.026710,0.102209,0.000012,185,...,457.847565,0.374308,1374,1823,317,52.306202,8,8019,80190247,whisper
4797,0.000064,0.000047,0.000102,0.168865,0.000027,1.036489e-08,0.043928,0.168865,0.000017,220,...,540.011383,0.373218,1628,2147,375,59.650508,8,8019,80190248,whisper
4798,0.000020,0.000014,0.000033,0.040123,0.000002,1.062261e-09,0.010436,0.040123,0.000005,166,...,410.998047,0.374330,1232,1644,285,46.464064,8,8019,80190249,whisper


In [2]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [3]:
def lda():
    ids = [1001,1002,1003,2001,2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032,4001,4002,7001,7002,8002,8003, 8010, 8016, 8017, 8018, 8019]
    scores = np.array([])
    for x in range (len(ids)):
        test_ids = []
        test_ids = ids[x:x+1]
        train_ids = np.delete(ids,x)
        #print (test_ids)
        X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
        k_clf = LinearDiscriminantAnalysis()
        k_clf.fit(X_train, y_train)
        score = k_clf.score(X_test, y_test)
        sscore = np.array([score])
        scores = np.concatenate([scores,sscore],axis=0)
        print("Accuracy for session " + str(test_ids) +" is "+str(score))
    print ("Durchschnitt: ")
    print((scores.mean()))

In [4]:
def lda_user1_session1():
    train_ids = [1002,1003]
    test_ids = [1001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score
    
def lda_user1_session2():
    train_ids = [1001,1003]
    test_ids = [1002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score
    
def lda_user1_session3():
    train_ids = [1001,1002]
    test_ids = [1003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 3 is "+str(score))
    return score
    
def lda_user2_session1():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score  
    
def lda_user2_session3():
    train_ids = [2001, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 3 is "+str(score))
    return score 
    
def lda_user2_session4():
    train_ids = [2003, 2001, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2004]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 4 is "+str(score))
    return score    
    
def lda_user2_session5():
    train_ids = [2003, 2004, 2001, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2005]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 5 is "+str(score))
    return score  
    
def lda_user2_session6():
    train_ids = [2003, 2004, 2005, 2001, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2006]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 6 is "+str(score))
    return score   
    
def lda_user2_session7():
    train_ids = [2003, 2004, 2005, 2006, 2001, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2007]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 7 is "+str(score))
    return score     
    
def lda_user2_session8():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2001, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2008]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 8 is "+str(score))
    return score 
    
def lda_user2_session9():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2001, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2009]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 9 is "+str(score))
    return score   
    
def lda_user2_session10():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2001, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2010]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 10 is "+str(score))
    return score     
    
def lda_user2_session12():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2001, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2012]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 12 is "+str(score))
    return score 
    
def lda_user2_session13():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2031, 2032] 
    test_ids = [2013]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 13 is "+str(score))
    return score   
    
def lda_user2_session28():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2013, 2029, 2031, 2032] 
    test_ids = [2028]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 28 is "+str(score))
    return score  
    
def lda_user2_session29():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2013, 2031, 2032] 
    test_ids = [2029]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 29 is "+str(score))
    return score     
    
def lda_user2_session31():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2013, 2032] 
    test_ids = [2031]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 31 is "+str(score))
    return score  
    
def lda_user2_session32():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2031, 2013] 
    test_ids = [2032]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 32 is "+str(score))
    return score 
    
def lda_user4_session1():
    train_ids = [4002]
    test_ids = [4001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score   
    
def lda_user4_session2():
    train_ids = [4001]
    test_ids = [4002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score 
    
def lda_user7_session1():
    train_ids = [7002]
    test_ids = [7001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score 
    
def lda_user7_session2():
    train_ids = [7001]
    test_ids = [7002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score 
    
def lda_user8_session2():
    train_ids = [8003, 8010, 8016, 8017, 8018, 8019]
    test_ids = [8002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score   
    
def lda_user8_session3():
    train_ids = [8002, 8010, 8016, 8017, 8018, 8019]
    test_ids = [8003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 3 is "+str(score))
    return score  
    
def lda_user8_session10():
    train_ids = [8003, 8002, 8016, 8017, 8018, 8019]
    test_ids = [8010]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 10 is "+str(score))
    return score   
    
def lda_user8_session16():
    train_ids = [8003, 8010, 8002, 8017, 8018, 8019]
    test_ids = [8016]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 16 is "+str(score))
    return score    
    
def lda_user8_session17():
    train_ids = [8003, 8010, 8016, 8002, 8018, 8019]
    test_ids = [8017]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 17 is "+str(score))
    return score  
    
def lda_user8_session18():
    train_ids = [8003, 8010, 8016, 8017, 8002, 8019]
    test_ids = [8018]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 18 is "+str(score))
    return score     
    
def lda_user8_session19():
    train_ids = [8003, 8010, 8016, 8017, 8018, 8002]
    test_ids = [8019]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 

In [5]:
def lda_all():
    print ("User 1 : " )
    a = lda_user1_session1()
    b = lda_user1_session2()
    c = lda_user1_session3()
    print ("User 2 : " )
    d = lda_user2_session1()
    e = lda_user2_session3()
    f = lda_user2_session4()
    g = lda_user2_session5()
    h = lda_user2_session6()
    i = lda_user2_session7()
    j = lda_user2_session8()
    k = lda_user2_session9()
    l = lda_user2_session10()
    m = lda_user2_session12()
    n = lda_user2_session13()
    o = lda_user2_session28()
    p = lda_user2_session29()
    q = lda_user2_session31()
    r = lda_user2_session32()
    print ("User 4 : " )
    s = lda_user4_session1()
    t = lda_user4_session2()
    print ("User 7 : " )
    u = lda_user7_session1()
    v = lda_user7_session2()
    print ("User 8 : " )
    w = lda_user8_session2()
    x = lda_user8_session3()
    y = lda_user8_session10()
    z = lda_user8_session16()
    aa = lda_user8_session17()
    bb = lda_user8_session18()
    cc = lda_user8_session19()
    scores = np.array([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,p,q,r,s,t,u,v,w,x,y,z,aa,bb,cc])
    print("Durchschnitt:")
    print((scores.mean()))


In [6]:
lda_all()

User 1 : 
Accuracy for session 1 is 0.6066666666666667
Accuracy for session 2 is 0.66
Accuracy for session 3 is 0.6466666666666666
User 2 : 
Accuracy for session 1 is 0.3333333333333333
Accuracy for session 3 is 0.6466666666666666
Accuracy for session 4 is 0.34
Accuracy for session 5 is 0.4
Accuracy for session 6 is 0.5466666666666666
Accuracy for session 7 is 0.48
Accuracy for session 8 is 0.48
Accuracy for session 9 is 0.4066666666666667
Accuracy for session 10 is 0.48
Accuracy for session 12 is 0.47333333333333333
Accuracy for session 13 is 0.24
Accuracy for session 28 is 0.4
Accuracy for session 29 is 0.32666666666666666
Accuracy for session 31 is 0.38666666666666666
Accuracy for session 32 is 0.37333333333333335
User 4 : 
Accuracy for session 1 is 0.32
Accuracy for session 2 is 0.38
User 7 : 
Accuracy for session 1 is 0.3333333333333333
Accuracy for session 2 is 0.30666666666666664
User 8 : 
Accuracy for session 2 is 0.5066666666666667
Accuracy for session 3 is 0.47333333333333333